In [164]:
import glob
import numpy as np
from scipy import misc
import shutil
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
# import skfuzzy

In [170]:
all_path=glob.glob("D:/Project/Train/*/*Train*")

# categories=get_categories(all_path)
CLUSTER_SIZE=3
paths=all_path
categories_object=Categories(all_path)
categories=categories_object.get_categories()
categories = list(categories)
for category in categories:
    paths=glob.glob("D:/Project/Train\\"+category+"\\*Train*")
    image_array=create_image_np_array(paths)
    model=Model(image_array,CLUSTER_SIZE)
    kmeans=model.model_fitting()
    predicted_array=model_predict(kmeans,image_array)
    copy=Copy(CLUSTER_SIZE,predicted_array,paths,category)
    copy.transfer()

    

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  after removing the cwd from sys.path.


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'D:/Denim/'

In [162]:
class Categories:
    def __init__(self, path):
        self.path = path
    def get_categories(self):
        categories=[]
        for i in range(len(self.path)):
            categories.append(self.path[i][findnth(self.path[i],"\\",1)+1:findnth(self.path[i],"\\",2)])

        categories=set(categories)
        return categories

In [163]:
class Sub_categories(Categories):
    def __init__(self, path):
        super().__init__(path)
    def get_sub_categories(path):
        sub_categories=[]
        for i in range(len(self.path)):
            sub_categories.append(self.path[i][findnth(self.path[i],"\\",2)+1:findnth(self.path[i],"\\",3)])

        sub_categories=set(sub_categories)
        return sub_categories

In [70]:
def resize_image(image_array):
    scale_percent = 60 # percent of original size
    width = int(image_array.shape[1] * scale_percent / 100)
    height = int(image_array.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv2.resize(image_array, dim, interpolation = cv2.INTER_AREA) 
    return resized

In [59]:

def create_image_np_array(paths):
    images=[]
    for path in paths:
        original_image=misc.imread(path)
        resized_image=resize_image(original_image)
        images.append(resized_image.flatten().tolist())
    return np.array(images)

In [166]:
class Model:
    def __init__(self, image_array,cluster_size):
        self.image_array=image_array
        self.cluster_size=cluster_size
    def model_fitting(self):
        kmeans = KMeans(n_clusters=self.cluster_size, n_init=1000, random_state=0)
        return kmeans.fit(self.image_array)

In [50]:
def model_predict(kmeans,image_array):
    predicted_array=kmeans.predict(image_array)
    return predicted_array

In [28]:
############ to find the nth letter ###############
def findnth(haystack, needle, n):
    parts= haystack.split(needle, n)
    if len(parts)<=n:
        return -1
    return len(haystack)-len(parts[-1])-len(needle)

In [169]:
class Copy:
    def __init__(self, cluster_size,predicted_array,source,category):
        self.cluster_size=cluster_size
        self.predicted_array=predicted_array
        self.source=source
        self.category=category
    def transfer(self):
        path_new="D:/"+self.category+"/"
        os.mkdir(path_new)
        for i in range(self.cluster_size):   
            os.mkdir(path_new+str(i))
        for i,j in enumerate(self.predicted_array):
            src_file=self.source[i]
            dest=path_new+str(j)
            shutil.copy(src_file, dest)